In [18]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
#import pandas_profiling

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

## FE

In [15]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_196,var_197,var_198,var_199,mean,std,min,max,sum,median
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,7.8784,8.5635,12.7803,-1.0914,7.281591,9.308182,-21.4494,43.1127,1456.3182,6.77040
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,8.1267,8.7889,18.3560,1.9518,7.076818,10.310257,-47.3797,40.5632,1415.3636,7.22315
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,-6.5213,8.2675,14.7222,0.3965,6.204483,8.731476,-22.4038,33.8820,1240.8966,5.89940
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,-2.9275,10.2922,17.9697,-8.9996,6.441160,9.570048,-35.1659,38.1015,1288.2319,6.70260
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,3.9267,9.5031,17.9974,-8.8104,6.771155,11.258868,-65.4863,41.1037,1354.2310,6.94735


In [14]:

var_list = [f for f in train_df.columns if 'var' in f]

train_df['mean'] = np.mean(train_df[var_list],axis=1)
train_df['std'] = np.std(train_df[var_list],axis=1)
train_df['min'] = np.min(train_df[var_list],axis=1)
train_df['max'] = np.max(train_df[var_list],axis=1)
train_df['sum'] = np.sum(train_df[var_list],axis=1)
train_df['median'] = np.median(train_df[var_list],axis=1)

## Model

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_folds = 5
random_seed = 26
model = 'lgb_simple_stats'


model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: lgb_simple_stats_5_folds


In [19]:
clfs = []
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
oof_preds = np.zeros((len(train_df), 1))
test_preds = np.zeros((len(test_df), 1))

X = train_df.iloc[:,2:]
y = train_df['target']
X_test = test_df.iloc[:,1:]
test_ids = test_df.ID_code.values


parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    #'scale_pos_weight': 400,
    #'device' : 'gpu' ,
    'boosting': 'gbdt',
    'num_leaves': 31, #31
    'feature_fraction': 0.5,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}


for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
    print("Current Fold: {}".format(fold_+1))
    trn_x, trn_y = X.iloc[trn_, :], y[trn_]
    val_x, val_y = X.iloc[val_, :], y[val_]


    trn_lgb = lgb.Dataset(trn_x, trn_y)
    val_lgb = lgb.Dataset(val_x, val_y)
    clf = lgb.train(parameters,
                         train_set=trn_lgb,
                         #valid_sets=[valid_data_lgb,holdout_data_lgb],
                         valid_sets=[trn_lgb, val_lgb],
                         num_boost_round=3000,
                         early_stopping_rounds=50,
                         verbose_eval=500)
    


    val_pred = clf.predict(val_x)
    test_fold_pred = clf.predict(X_test)

    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))
    
    
test_preds /= n_folds
roc_score = roc_auc_score(y, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))


print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/{}_{}.csv'.format(model_name,str(roc_score)), index=False)



Current Fold: 1
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.962189	valid_1's auc: 0.888883
Early stopping, best iteration is:
[710]	training's auc: 0.974483	valid_1's auc: 0.891514
AUC = 0.8915141158662927
Current Fold: 2
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.962462	valid_1's auc: 0.887888
Early stopping, best iteration is:
[706]	training's auc: 0.974578	valid_1's auc: 0.889815
AUC = 0.8898147184092875
Current Fold: 3
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.96226	valid_1's auc: 0.892379
Early stopping, best iteration is:
[653]	training's auc: 0.971556	valid_1's auc: 0.893959
AUC = 0.8939590264353607
Current Fold: 4
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.962451	valid_1's auc: 0.892146
Early stopping, best iteration is:
[846]	training's auc: 0.98061	valid_1's auc: 0.894311
AUC = 0.8943109169742777
Current Fo

NameError: name '__file__' is not defined

In [43]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [44]:
shutil.copyfile(os.path.basename(NOTEBOOK_FULL_PATH), 
                             '../models/{}_{}.py'.format(model_name, str(roc_score)))

'../models/lgb_simple_stats_5_folds_0.892613793715947.py'

<IPython.core.display.Javascript object>